<a href="https://colab.research.google.com/github/neelkanthrawat/QBM_sept2022/blob/main/qbm_codes/qmcmc_trial_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install qiskit ipywidgets # while working on google colab!

In [ ]:
#!wget https://raw.githubusercontent.com/neelkanthrawat/QBM_sept2022/main/qbm_codes/qbm_utils.py# while working on google colab, run this cell

In [1]:
from qbm_utils import *

In [2]:
# define the model
## put n_spins even always
n_spins = 4

## construct problem Hamiltonian ##
shape_of_J=(n_spins,n_spins)
J =  np.random.randn(n_spins,n_spins)#np.random.uniform(low= -1, high= 1, size= (n_spins, n_spins) )
J = 0.5 * (J + J.transpose() )
h = np.random.randn(n_spins)#np.random.uniform(low= -1, high = 1, size= (n_spins))

# instantiate the model
model = IsingEnergyFunction(J, h)

alpha = np.sqrt(n_spins) / np.sqrt( sum([J[i][j]**2 for i in range(n_spins) for j in range(i)]) + sum([h[j]**2 for j in range(n_spins)])  )


In [4]:
model.get_partition_sum()

RecursionError: maximum recursion depth exceeded while calling a Python object

In [3]:
# symmetric uniform transition matrix for uniform transition rule
num_spins=4
num_elems=2**(num_spins)
transit_mat=np.ones((num_elems,num_elems))*(1./(2*num_elems))# in general case transit matrix could be entirely different
np.fill_diagonal(transit_mat, 1./2)

# Run MCMC with uniform random transition strategy
N_hops=50
dict_states_quantum_mcmc, mcmc_hist = quantum_enhanced_mcmc(N_hops, num_spins, num_elems, model, alpha, num_trotter_steps=10, return_history=True, return_last_n_states=50)

# get actual probability distribution
transits_bltz = dict( [ ( state, model.get_boltzmann_prob(state) ) for state in dict_states_quantum_mcmc.keys() ] )
# plot actual prob distribution as well as one obtained from MCMC
plot_histogram( [transits_bltz, dict_states_quantum_mcmc] , figsize=(15, 7), legend= ['model', 'Quantum-enhanced MCMC' ])#, sort= 'value_desc')

starting with:  1111


100%|██████████| 50/50 [00:41<00:00,  1.22it/s]


RecursionError: maximum recursion depth exceeded while calling a Python object

In [7]:
dict_states_quantum_mcmc

Counter({'1011': 3,
         '1101': 10,
         '1111': 23,
         '1010': 3,
         '1110': 3,
         '1001': 3,
         '0000': 2,
         '1000': 3})

In [8]:
import itertools

In [18]:
np.array(list(itertools.product([1,0], repeat= 3)))

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [1, 0, 0],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 0]])

In [7]:
l = [1,2,3]
l[-5:]

[1, 2, 3]

In [ ]:
initialise_qc(2, '11')[0].qubits

[Qubit(QuantumRegister(2, 'spin'), 0), Qubit(QuantumRegister(2, 'spin'), 1)]

In [ ]:
observables= ['rejection_count']
observable_dict = dict([ ( elem, []  ) for elem in observables ])

In [ ]:
observable_dict['rejection_count'].append(0.2)

In [ ]:
observable_dict['rejection_count']

[0.1, 0.2]

In [ ]:
observable_df = pd.DataFrame(observable_dict)
# observable_df.append([[0.4]])

In [ ]:
observable_df

,rejection_count
0,0.1
1,0.2


In [ ]:
## class for updating; 
# requirements ->
# update_strategy = {classical, quantum}
# measurements = {} : list of observables to measure over iterations



def run_mcmc_sampling():
    